In [8]:
!pip install tensorflow --user

  Using cached tensorflow-2.12.0-cp39-cp39-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.12.0-cp39-cp39-win_amd64.whl (272.8 MB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached jax-0.4.10-py3-none-any.whl
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached grpcio-1.54.2-cp39-cp39-win_amd64.whl (4.1 MB)
  Using cached tensorboard-2.12.3-py3-none-any.whl (5.6 MB)
  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Using cached ml_dtypes-0.1.0-cp39-cp39-win_amd64.whl (120 kB)
  Using cached google_auth-2.18.1-py2.py3-none-any.whl (178 kB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl (18 kB)
  Using cached cachetools-5.3.0-py3-none-any.whl (9.3 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [17]:
from scipy.sparse import load_npz
import pandas as pd
df_preprocessed_general = load_npz('./ds_salaries_BoWPreprocessing.npz')
df_target = pd.read_csv("./ds_salaries_target.csv")

kfold = KFold(n_splits=10, shuffle=True, random_state=42)

In [21]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

# Define the input dimension based on your data
input_dim = 280

# Define your preprocessed data and target variable
df_preprocessed_general = df_preprocessed_general
df_target = df_target

# Modell erstellen 
model = tf.keras.Sequential() 
model.add(layers.Dense(64, activation='relu', input_shape=(input_dim,)))

# Compile the model
model.compile(optimizer='adam', loss='mse')  # Set the appropriate optimizer and loss function

steps = [('estimator', model)]  
pipeline = Pipeline(steps)  
r2_scores = cross_val_score(pipeline, df_preprocessed_general, df_target.values.ravel(), cv=kfold, scoring='r2') 
mse_scores = -cross_val_score(pipeline, df_preprocessed_general, df_target.values.ravel(), cv=kfold, scoring='neg_mean_squared_error')  

print("General Preprocessing:") 
print("ANNs MLP Regressor R2: ", np.mean(r2_scores)) 
print("ANNs MLP Regressor MSE: ", np.mean(mse_scores))

ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Paul\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Paul\anaconda3\lib\site-packages\sklearn\pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\Paul\AppData\Roaming\Python\Python39\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "C:\Users\Paul\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\framework\ops.py", line 7262, in raise_from_not_ok_status
    rais

In [23]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

#Modell erstellen
model = tf.keras.Sequential()
model.add(layers.Dense(10, activation='relu', input_shape=(input_dim,)))
model.add(layers.Dense(1))  # Annahme: Regressionsmodell mit einer Ausgabegröße

#Modell kompilieren
model.compile(optimizer='adam', loss='mean_squared_error')

#Estimator für TensorFlow-Modell
estimator = tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=lambda: model, epochs=10, batch_size=32, verbose=0)

steps = [('estimator', estimator)]
pipeline = Pipeline(steps)

#Kreuzvalidierung und Bewertung
r2_scores = cross_val_score(pipeline, df_preprocessed_general, df_target.values.ravel(), cv=kfold, scoring='r2')
mse_scores = -cross_val_score(pipeline, df_preprocessed_general, df_target.values.ravel(), cv=kfold, scoring='neg_mean_squared_error')

print("General Preprocessing:")
print("ANNs MLP Regressor R2: ", np.mean(r2_scores))
print("ANNs MLP Regressor MSE: ", np.mean(mse_scores))

C:\Users\Paul\AppData\Local\Temp\ipykernel_1728\3129219867.py:15: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=lambda: model, epochs=10, batch_size=32, verbose=0)


ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Paul\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Paul\anaconda3\lib\site-packages\sklearn\pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\Paul\AppData\Roaming\Python\Python39\site-packages\keras\wrappers\scikit_learn.py", line 175, in fit
    history = self.model.fit(x, y, **fit_args)
  File "C:\Users\Paul\AppData\Roaming\Python\Python39\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_t

In [31]:
import tensorflow as tf
print(tf.__version__)
import numpy as np
import matplotlib.pyplot as plt

2.12.0


In [32]:
tf.random.set_seed(42)  #first we set random seed
model = tf.keras.Sequential([
                             tf.keras.layers.Dense(1)
])

model.compile(loss = tf.keras.losses.mae, #mae stands for mean absolute error
              optimizer = tf.keras.optimizers.SGD(), #stochastic GD
              metrics = ['mae'])
model.fit( df_preprocessed_general, df_target, epochs = 10)

ValueError: Failed to find data adapter that can handle input: <class 'scipy.sparse._csr.csr_matrix'>, <class 'pandas.core.frame.DataFrame'>